# **Advanced regression and prediction final project**: data preprocessing

### Daniel A.
### UID: 100444499

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import json

### Importing data

In [2]:
# Importing the raw data
raw_data = pd.read_csv('../data/raw/wb_raw_data.csv')

### Initial modifications and selection of range of rows

In [3]:
# excluding aggregates
codes_to_exclude = raw_data.iloc[5497:5544,3].values

# filtering the dataset
data = raw_data[~(raw_data['Country Code'].isin(codes_to_exclude))] 

# removing final diagnostic columns
data = data[~(data['Time'].isna()) & ~(data['Time Code'].isna())]

# converting year column to integer
data['Time'] = data['Time'].astype(int)

# replacing .. with NAN as the raw data indends this to be a NAN
data = data.replace('..',np.nan)

# sorting values
data = data.sort_values(['Time','Country Name'])

#### Years imported

In [4]:
# checking years we have queried
years = data.Time.unique()
print(years)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020]


### Filling NAs with previous year's data

In [5]:
# making a dictionary with the subsets of the main dataframe
dfs = {}
for year in years:
    dfs[year] = data[data['Time'] == year].reset_index(drop=True)

# replacing nans in the 2020 dataframe with previous years data, as 
# the previous years' data still serves us a purpose for the analysis
for year in years:   
    dfs[2020] = dfs[2020].fillna(dfs[year])

### Importing and cleaning up the COVID-19 dataset

In [6]:
# importing the data
covid = pd.read_csv('../data/raw/owid-covid-data-1.csv')

# excluding rows without continent and matching a specific date
covid = covid[(~covid['continent'].isna()) & (covid['date'] == '2021-05-01')]

# only taking the columns we're interested in
covid = covid[['iso_code', 'total_cases_per_million', 'total_deaths_per_million', 'total_tests_per_thousand', 'total_vaccinations_per_hundred', 'population','population_density', 'median_age', 'human_development_index']]

# joining the COVID-19 and the development indicators datasets
dfs[2020] = dfs[2020].merge(right=covid,  left_on='Country Code', right_on='iso_code', how='inner')
dfs[2020]

,Time,Time Code,Country Name,Country Code,Access to electricity (% of population),Bank liquid reserves to bank assets ratio (%),"Birth rate, crude (per 1,000 people)",Gini index (World Bank estimate),"Life expectancy at birth, total (years)",Exports of goods and services (% of GDP),...,"International tourism, number of departures",iso_code,total_cases_per_million,total_deaths_per_million,total_tests_per_thousand,total_vaccinations_per_hundred,population,population_density,median_age,human_development_index
0,2020,YR2020,Afghanistan,AFG,23,18.909954522551,48.021,NaN,55.841,NaN,...,NaN,AFG,1539.727,67.586,NaN,NaN,38928341.0,54.422,18.6,0.511
1,2020,YR2020,Albania,ALB,99.9,28.1796966161027,16.436,31.7,73.955,20.1153637600108,...,955000,ALB,45585.169,832.580,214.995,17.17,2877800.0,104.871,38.0,0.795
2,2020,YR2020,Algeria,DZA,98.7472686767578,34.4854290989948,19.554,27.6,70.64,42.0697279237388,...,1006000,DZA,2789.238,74.365,NaN,NaN,43851043.0,17.348,29.1,0.748
3,2020,YR2020,Andorra,AND,100,NaN,11.3,NaN,NaN,NaN,...,NaN,AND,171254.773,1617.809,NaN,NaN,77265.0,163.755,NaN,0.868
4,2020,YR2020,Angola,AGO,20,24.0002906144722,48.15,52,46.522,89.6858311851382,...,NaN,AGO,815.882,18.256,NaN,NaN,32866268.0,23.890,16.8,0.581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,2020,YR2020,Vietnam,VNM,87.187873840332,NaN,17.429,37,73.025,53.9214970086223,...,NaN,VNM,30.224,0.360,NaN,NaN,97338583.0,308.127,32.6,0.704
193,2020,YR2020,West Bank and Gaza,PSE,99.7,10.466392868065,40.9,34,71.022,20.5165059347181,...,NaN,PSE,58113.669,636.882,NaN,NaN,5101416.0,778.202,20.4,0.708
194,2020,YR2020,"Yemen, Rep.",YEM,49.9811706542969,NaN,39.517,34.7,60.683,NaN,...,NaN,YEM,212.064,41.206,NaN,NaN,29825968.0,53.508,20.3,0.470
195,2020,YR2020,Zambia,ZMB,16.7,23.145629240221,45.271,42.1,44,23.9238608307712,...,NaN,ZMB,4986.413,68.048,76.520,NaN,18383956.0,22.995,17.7,0.584


### Removing columns with more than 35 NAs

In [7]:
# removing columns where there's too many NANs
cols_to_keep = []
df = {}
for col,val in zip(dfs[2020].columns,dfs[2020].isna().sum()):
    if val < 35:
        cols_to_keep.append(col)
df = dfs[2020][cols_to_keep]

In [8]:
# dropping repeated iso code and repetitive columns
df = df.drop(['iso_code', 'Foreign direct investment, net (BoP, current US$)'], axis=1)

# Columns kept from the finalized dataset
df.columns

Index(['Time', 'Time Code', 'Country Name', 'Country Code',
       'Access to electricity (% of population)',
       'Birth rate, crude (per 1,000 people)',
       'Life expectancy at birth, total (years)',
       'Exports of goods and services (% of GDP)',
       'Compulsory education, duration (years)',
       'GDP per capita, PPP (current international $)',
       'Individuals using the Internet (% of population)',
       'Mortality rate, infant (per 1,000 live births)',
       'Inflation, consumer prices (annual %)',
       'Consumer price index (2010 = 100)',
       'Crop production index (2014-2016 = 100)',
       'Goods imports (BoP, current US$)',
       'Prevalence of anemia among pregnant women (%)',
       'Diabetes prevalence (% of population ages 20 to 79)',
       'Human capital index (HCI) (scale 0-1)',
       'Adjusted net enrollment rate, primary (% of primary school age children)',
       'Immunization, measles (% of children ages 12-23 months)',
       'CO2 emissions

### Removing countries with more than 4 NAs

In [9]:
# checking which countries have the most NANs
all_countries = df['Country Name'].values
countries_removed = []
for country,val in zip(all_countries,df.isna().sum(axis=1)):
    # removing at 3 nans per row
    if val > 4:
        countries_removed.append(country)

# finally filtering to remove them
# either way, these countries are mostly dependencies or 
# complex countries to get data from, like North Korea
# so even after imputing, this would probably
# yield unrealistic values
df = df[~(df['Country Name'].isin(countries_removed))]

### Fixing columns' types and names

In [10]:
# modifying columns that are supposed to be numeric
cols = list(set(list(df.columns)) - set(['index', 'Time', 'Time Code', 'Country Name', 'Country Code']))
for col in cols:
    df[col] = df[col].astype(float)

# new column names
colnames = ['year','year_code','country_name',
            'country_code','access_to_electricity',
            'birth_rate','life_expectancy',
            'exports_perc_gdp','education_years','gdp_per_capita_ppp',
            'perc_internet_users','infant_mort_rate',
            'inflation_perc', 'consumer_price_index', 'crop_production_index',
            'goods_imports', 'prevalence_of_anemia_pregnancy', 'diabetes_prevalence',
            'human_capital_index', 'net_enrollment_rate', 'measles_immunization',
            'co2_emissions_gaseous_fuel', 'fossil_fuel_energy_consumption', 'fuel_exports', 'fuel_imports',
            'investment_inflows', 'investment_outflows', 'mobile_subscriptions',
            'agricultural_land', 'greenhouse_gas_em', 'age_dependency_ratio', 
            'imports_annual_growth', 'int_tourism_arrivals', 'total_covid_cases_per_million',
            'total_covid_deaths_per_million', 'population', 'population_density', 'median_age',
            'human_development_index']

# original colnames : colnames dict
col_dict = dict(zip(df.columns,colnames))

# renaming columns as the names are too long
df.columns = colnames

### Constructing the target variable: Access to and reliance on technology

In [11]:
# converting mobile subscriptions to a percentage
df['mobile_subscriptions'] = ((df['population']/100 * df['mobile_subscriptions'])/df['population'])

# creating ar_tech
df['ar_tech'] = df['mobile_subscriptions']*0.10 + (df['perc_internet_users']/100)*0.35 + (df['access_to_electricity']/100)*0.55

### Exporting the data from Python into R to impute the few missing values left

In [12]:
# dropping population from the dataset as it won't be useful for our purposes
df = df.drop(['population'], axis=1)

# saving the dataset before imputing
df.to_csv('../data/preproc_outputs/data_before_imp.csv')

# dumping the col information into a json
with open('../data/preproc_outputs/columns.json','w') as f:
    f.write(json.dumps(col_dict))

In [1]:
# importing mice library to impute the data
library(mice)

# reading the csv outputted earlier in the IRkernel to impute the data
df <- read.csv('../data/preproc_outputs/data_before_imp.csv')
df <- df[,2:length(names(df))]

# instantiating the imputer and imputing the data
imp = mice(df, m = 5, method = "cart")
df = complete(imp)

# saving csv for reference
write.csv(df,'../data/data.csv')


Attaching package: ‘mice’


The following object is masked from ‘package:stats’:

    filter


The following objects are masked from ‘package:base’:

    cbind, rbind





 iter imp variable
  1   1  exports_perc_gdp  education_years  gdp_per_capita_ppp  inflation_perc  consumer_price_index  goods_imports  prevalence_of_anemia_pregnancy  human_capital_index  net_enrollment_rate  measles_immunization  fossil_fuel_energy_consumption  fuel_exports  fuel_imports  investment_outflows  greenhouse_gas_em  age_dependency_ratio  imports_annual_growth  int_tourism_arrivals  total_covid_deaths_per_million  population_density  median_age
  1   2  exports_perc_gdp  education_years  gdp_per_capita_ppp  inflation_perc  consumer_price_index  goods_imports  prevalence_of_anemia_pregnancy  human_capital_index  net_enrollment_rate  measles_immunization  fossil_fuel_energy_consumption  fuel_exports  fuel_imports  investment_outflows  greenhouse_gas_em  age_dependency_ratio  imports_annual_growth  int_tourism_arrivals  total_covid_deaths_per_million  population_density  median_age
  1   3  exports_perc_gdp  education_years  gdp_per_capita_ppp  inflation_perc  consumer_price

Warning message:
“Number of logged events: 529”
